# **Este *ipynb* está enmarcado dentro de un trabajo que busca predecir qué alumnos del departamento de Sistemas la UTN FRBA desertarán.**

Los datos disponibles se presentan en tres archivos separados.

Los registros de estas tres tablas se encuentran relacionados a través de un ID anonimizado que corresponde a un estudiante en particular. 

En este **ipynb** se incorpora la feature a predecir '**deserto**' y se define el valor que toma esta para los datos disponibles.


## **Aclaraciones**

Aquellas líneas de código identificadas con **" # "** fueron extraídas de la siguente fuente:
* https://github.com/sebajarem/Analisis_desercion_en_ingenieria/blob/master/desercion/munge/00_datos_01/Desertores%20corregidos.ipynb

Aquellas líneas de código identificadas con **" ## "** son de elaboración propia.

---

## **Definiciones**

* **Actividad​**: Se considera que un estudiante tiene ​actividad en un año determinado si: tiene alguna materia con el campo “Tipo de aprobación” distinto de “libre” **ó** si rindió algún final de esa u otra materia. 
 *Notar que ​actividad es una característica siempre asociada a un año determinado ​XX​.*
---
* (ALF) **Año Lectivo Final**​: Año que se toma como punto de partida para analizar HACIA ATRÁS las historias disponibles hasta 2008. En nuestro caso, por ahora, **ALF = 2016**. 
---
* (ALI) **Año Lectivo Inicial**: Año a partir de que contamos con datos. En nuestro caso, por ahora; **ALI = 2008**. 
---
* (AUA) **Año de Última Actividad​**: Último año en que el estudiante haya tenido alguna actividad​. 
---
* (AI) **Año de Ingreso**​: Año en el que el estudiante haya realizado su primera actividad.
---
* **Alumno ​activo a la fecha ​XX**​: Se considera que un estudiante está “activo a la fecha ​XX​” si en el año ​XX ​registró ​actividad​.  
---
* (AE) **Año de egreso**​: Año en que aprobó “Proyecto Final”.  
---
* **Condición de Egresado​**: La condición de ​egresado se puede asignar a un alumno en el año de egreso o posteriores. *Ver que se necesita indicar un año para ver si pertenece a esa categoría.* 
---
* **Condición de Desertor**​: Se considera que un alumno es desertor para el año ​XX si **no** registra actividad ni en el año ​XX ni en el año ​XX - 1​. *Ver que se necesita indicar un año para ver si pertenece a esa categoría.*
---

## **Importación de librerías**





In [ ]:
## Importamos librerías para manipulación de datos.
import pandas as pd
import numpy as np

## **Defino años para el análisis**

In [ ]:
ALF = 2016 ## Año lectivo final
ALI = 2008 ## Año lectivo inicial

#Seteo el ano sobre el que voy a medir
ano = 2017

## **Google Colaboratory o Local**
El notebook podrá ser corrido tanto localmente como en Google Colaboratory.

El usuario deberá modificar el root path de acuerdo a su conveniencia.


In [ ]:
## Verificamos si estamos corriendo el noteboock en Google Colaboratory.
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)

## En el caso de estar en Google Colab, montamos nuestro Drive.
if var_google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  ## Direccion root donde está el notebook.
  root_path = "/content/gdrive/MyDrive/Colab Notebooks/GIAR/"

## En el caso de no estar en Google Colab, estamos corriendo localmente el notebook.
else:
  root_path = ""

True
Mounted at /content/gdrive


## **Datasets**

In [ ]:
## Importamos el dataset Datos-Alumnos-SIGA.
alumnos = pd.read_csv(root_path + 'datos/Datos-Alumnos-SIGA.csv')

#Agrego la columna de desertor
alumnos['deserto']=0

In [ ]:
## Importamos el dataset Cursadas-Alumnos-SIGA.
cursadas = pd.read_csv(root_path + 'datos/Cursadas-Alumnos-SIGA.csv')

In [ ]:
## Importamos el dataset Finales-Alumnos-SIGA.
finales = pd.read_csv(root_path + 'datos/Finales-Alumnos-SIGA.csv')

##**DEFINO EL VALOR DE LAS ETIQUETAS PARA CADA REGISTRO**#
### 0 = No desertó
### 1 = Desertó

Seleccionamos los alumnos que egresaron, indicando el año en que lo hiciceron.
* (**AE**) **Año de egreso​**: Año en que aprobó “Proyecto Final”.

In [ ]:
# Genero un listado con los alumnos que egresaron y el año en el que lo hicieron (AE).
ano_egreso = finales[(finales['Materia'] == 'Proyecto Final') & (finales['Aprobado'] == 1)][['Codigo Alumno','Año']].reset_index()

Si egresó se lo categoriza como **“noDesertor”** con las características calculadas 
**hasta AUA-2** (en este caso el AUA coincide con AE) (siempre que **ALI < AUA-2**).

Se pasa al siguiente individuo (vuelve al paso 1 para el individuo siguiente). 

In [ ]:
# Genero una lista de los alumnos que egresaron en un periodo anterior a la fecha de chequeo
alumnos_egreso = ano_egreso[ano_egreso['Año'] <= (ALF)]['Codigo Alumno']

  ## NO se tendrán en cuenta a aquellos que hayan egresado despues de 2016. La base de datos tiene info 
  ## de solo hasta 2016, por ende, los datos que hacen referencia a años posteriores han de ser ERRÓNEOS.

In [ ]:
##
alumnos_egreso.shape

(344,)

Busco todos los alumnos y la **maxima fecha de final** entre los que **NO** estan entre los egresados.

A todos aquellos que cumplan con la condición "*Si no egresó y AUA < (ALF-1)*", se los categoriza como **“desertor”** con las características calculadas **hasta AUA inclusive**.

Se pasa al siguiente individuo, va al 
paso 1.

In [ ]:
# Filtro los alumnos de acuerdo al ultimo ano que rindieron finales
alumno_finales_ult=finales[finales['Año']<=(ALF)].groupby('Codigo Alumno', as_index = False)['Año'].max()
# Genero una lista con los alumnos cuyo ÚLTIMO FINAL fue antes el ciclo lectivo ALF-1 (antes de 2015) y NO egresaron.
alumnos_rindieron_finales=alumno_finales_ult[(alumno_finales_ult['Año']<(ALF-1)) & (~alumno_finales_ult['Codigo Alumno'].isin(alumnos_egreso))]['Codigo Alumno']

Llegado este punto, para aquellos alumnos que hayan quedado **SIN ETIQUETAR** sólo puede darse la situación de que su última actividad esté dentro de los últimos dos años, ALF y ALF-1. 

Hay dos alternativas para estos casos: 

**Alternativa 1**

Si AI > (ALF-2) se lo descarta y se pasa al siguiente individuo, paso 1;

(no podemos asegurar el resultado) (esto en rigor es parte del 
preprocesamiento y limpieza inicial) 

Sino se lo categoriza como “noDesertor” con las características 
calculadas hasta (ALF-2).

**Alternativa 2** 

Se lo descarta y se pasa al siguiente individuo, paso 1.

*(habrá que ver cuántos son para considerar descartarlos o no. Lo que sea más facil)*


In [ ]:
# Filtro los alumnos de acuerdo al ÚLTIMO año en el que cursaron una materia.
alumno_cursadas_ult=cursadas[cursadas['Ciclo Lectivo de Cursada']<=(ALF)].groupby('Codigo Alumno' ,as_index = False)['Ciclo Lectivo de Cursada'].max().reset_index()

In [ ]:
# Genero una lista con los alumnos cuya ÚLTIMA CURSADA fue antes el ciclo lectivo ALF-1 (antes de 2015) y NO egresaron.
alumnos_cursaron=alumno_cursadas_ult[(alumno_cursadas_ult['Ciclo Lectivo de Cursada']<(ALF-1)) & (~alumno_cursadas_ult['Codigo Alumno'].isin(alumnos_egreso))]['Codigo Alumno']

In [ ]:
alumnos_cursaron

32        204220
34        459546
36        785181
37        806880
38        865092
          ...   
6252    10002066
6259    10002558
6268    10003340
6277    10003368
6282    10003384
Name: Codigo Alumno, Length: 2655, dtype: int64

##**⁉⁉⁉⁉⁉**
DUDA: NO SERÍA MÁS PRECISO QUE SE LOS CATEGORICE COMO DESERTORES SI SU ULTIMA CURSADA **Y** FINAL FUESEN ANTES DE **ALF-1**.

OSEA AQUELLOS QUE ESTEN TANTO EN alumnos_cursaron COMO EN alumnos_rindieron_finales

??????????????????????????????????????????

Un && en lugar de un ||:

In [ ]:
# 
alumnos_desertores=pd.concat([alumnos_cursaron,alumnos_rindieron_finales])

In [ ]:
# Defino el valor de las etiquetas.
alumnos['deserto'][alumnos['Codigo Alumno'].isin(alumnos_desertores)]=1

<ipython-input-15-ad2c0daf518d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alumnos['deserto'][alumnos['Codigo Alumno'].isin(alumnos_desertores)]=1


In [ ]:
#
alumnos.deserto.value_counts()

0    5266
1    3069
Name: deserto, dtype: int64

In [ ]:
# Guardo el listado de alumnos
alumnos.to_csv(root_path + 'datos/alumnos_desertores.csv')